# Introduction

In this tutorial, we'll demonstrate how to leverage a sample dataset stored in Azure Cosmos DB to ground OpenAI models. We'll do this taking advantage of Azure Cognitive Search's vector similarity search functionality. In the end, we'll create an interatice chat session with the GPT-3.5 completions model to answer questions about Azure services informed by our dataset. This process is known as Retrieval Augmented Generation, or RAG.

This tutorial borrows some code snippets and example data from the Azure Cognitive Search Vector Search demo repository: https://github.com/Azure/cognitive-search-vector-pr/. 

# Installing required libraries and packages
First, let's start by installing the packages that we'll need later. Note that for `azure-search-documents`, you'll need to install a specific version from the preview azure sdk. 
- `numpy` - explorative data analysis
- `openai` - provides convenient access to OpenAI rest-api
- `azure-core` - provides shared primitives, abstractions, and helpers for modern Java Azure SDK client libraries
- `azure-cosmos` - global distributed, multi-model database that is used in a wide range of applications and use cases
- `azure-search-documents` (sdk) - provides secure information retrieval at scale over user-owned content in traditional and conversational search applications

In [1]:
%pip install numpy
%pip install openai==0.28
%pip install python-dotenv
%pip install azure-core
%pip install azure-cosmos
%pip install tenacity
%pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


# Importing libraries

In [1]:

import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

.env file contains the necessary configurations, keys and endpoints for azure cosmos db, cognitive search and open ai.

In [22]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "environment.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

# Azure Cosmos DB
cosmosdb_endpoint = config['cosmos_db_api_endpoint']
cosmosdb_key = config['cosmos_db_api_key']
cosmosdb_connection_str = config['cosmos_db_connection_string']
cosmos_db_container_id = config['cosmos_db_container_id']
cosmos_db_id = config['cosmos_db_id']

# Azure Cognitive Search
cog_search_endpoint = config['cognitive_search_api_endpoint']
cog_search_key = config['cognitive_search_api_key']
cognitive_search_index_name = config['cognitive_search_index_name']
cognitive_search_indexer_name = config['cognitive_search_indexer_name']

# OpenAI
openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

# Text embeddings
embeddings_deployment = config['openai_embeddings_deployment']
completions_deployment = config['openai_completions_deployment']

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at https://aka.ms/oai/access. Once you have access, complete the following steps:

- Create an Azure OpenAI resource following this quickstart: https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
- Deploy a `completions` and `embeddings` model 
    - For more information on `completions`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/completions
    - For more information on `embeddings`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings
- Copy the endpoint, key, deployment names for (embeddings model, completions model) into the config.json file.

# Load data and create embeddings <a class="anchor" id="loaddata"></a>
Here we'll load a sample dataset containing information about Wells Fargo. Then we'll user Azure OpenAI's `text-embedding-ada-002` model  to create vector embeddings from this data.

In [24]:
# Load sample json data file in read mode
data_file = open(file="payments_text_to_be_ingested.json", mode="r")
data = json.load(data_file)
data_file.close()

In [25]:
# Take a peek of data for sample
data[0]

{'id': '1',
 'content': '\ufeffPayables. Optimize your cash flow. Wells Fargo’s payables products help you streamline processes, better manage expenses, and improve information access. Our comprehensive solutions automate your accounts payable (A/P) process and can add speed and transparency to your transactions.. Capitalize on smart technology. Wells Fargo’s commitment to technology helps to get results – increased efficiency and usability for you and your company. Our solutions include:. Electronic Payables Solutions. Streamline your A/P process, reduce errors and fraud risk, and decrease payment-processing time and cost when you convert from paper to electronic payables.. Commercial Card. Exercise greater control of your disbursements with a single A/P solution for purchasing, travel and entertainment, and fleet expenses.. Controlled Disbursement. Make effective cash management decisions and streamline reconciliation with access to true daily funding totals and intra-day account inf

# Generate Embeddings
 Here, we generate embeddings from string of text which will be used to vectorize data and user input for interactions with Azure OpenAI.

In [26]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine=embeddings_deployment)
    embeddings = response['data'][0]['embedding']
    time.sleep(1.0) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings

In [27]:
# Create a sample embedding
sample_embeddings = generate_embeddings('how are you?')
print (sample_embeddings)

[-0.021260535344481468, -0.01146192941814661, -0.00033121882006525993, -0.035567622631788254, -0.011918406002223492, 0.017783811315894127, -0.015557709150016308, -0.016408130526542664, -0.024974875152111053, -0.020497657358646393, 0.019434630870819092, 0.005387044511735439, -0.010136272758245468, -0.012706296518445015, -0.0007335665868595243, -0.008629275485873222, 0.021673239767551422, -0.011712053790688515, 0.011687041260302067, -0.01435711421072483, -0.0017102224519476295, 0.009167042560875416, 0.0043021319434046745, -0.015044954605400562, -0.018396615982055664, -0.01544515322893858, 0.026638198643922806, -0.011124262586236, 0.013619248755276203, -0.0316406786441803, 0.004517863504588604, 0.011543219909071922, -0.013531705364584923, 0.010786594823002815, 0.012818851508200169, -0.0174836628139019, 0.0021557556465268135, -0.017008427530527115, 0.0196597408503294, -0.0024090062361210585, 0.02743859589099884, -0.008547985926270485, -0.00734739052131772, -0.008272849023342133, -0.0428962

In [28]:
# Generate embeddings for the content field in json

start = datetime.datetime.now()
for item in data:
    content = item['content']
    content_embeddings = generate_embeddings(content)
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'
end = datetime.datetime.now()

print("Took {} seconds to generate embeddings...".format(end-start))

# Save embeddings to a file (for further use)
with open("text-sample_w_embeddings.json", "w") as f:
    json.dump(data, f)

Took 0:00:11.061439 seconds to generate embeddings...


# Create and Upload data to Azure Cosmos DB NoSQL resource
Let's start by creating an Azure Cosmos DB NoSQL Resource following this quick start guide: https://learn.microsoft.com/azure/cosmos-db/nosql/quickstart-portal

Then copy the connection details (endpoint, key, and connection string) into the .env file.

![Azure Cosmos DB and Container](https://learn.microsoft.com/en-us/azure/cosmos-db/media/account-databases-containers-items/cosmos-entities.png)

In [29]:
# Create the cosmos client to interact with the Azure Cosmos DB resource
client = CosmosClient(cosmosdb_endpoint, cosmosdb_key)

# Create a database in Azure Cosmos DB.
try:
    database = client.create_database_if_not_exists(id=cosmos_db_id)
    print(f"Database created: {database.id}")

except exceptions.CosmosResourceExistsError:
    print("Database already exists.")

Database created: OpsBotCosmosDB


In [30]:
# Create a container in Azure Cosmos DB.
try:
    partition_key_path = PartitionKey(path="/id")
    container = database.create_container_if_not_exists(
        id=cosmos_db_container_id,
        partition_key=partition_key_path,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}") 

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: OpsBotCosmosDBContainer


In [31]:
# Ingest embedded data into Cosmos DB container
for data_item in data:
    try:
        container.create_item(body=data_item)
    
    except exceptions.CosmosResourceExistsError:
        print("Data item already exists.")

![Our Own Azure Cosmos DB and Container](./assets/MyAzureCosmosDBContainer.png)
Here, we can see the database and containers created, and data also ingested.

## Azure Cognitive Search <a class="anchor" id="cognitivesearch"></a>

Next, let's create an Azure Cognitive Search resource following this quick start: https://learn.microsoft.com/azure/search/search-create-service-portal

Now copy the connection details (endpoint, key, and connection string) into the .env file.

## Create a search index in Cognitive Search 

Let's create the Search Index over all required fields we have in our Azure Cosmos DB collection. Learn more about Azure Cognitive Search Indexes [here](https://learn.microsoft.com/azure/search/search-how-to-create-search-index).

In [32]:
# Create index

cog_search_cred = AzureKeyCredential(cog_search_key)
index_name = cognitive_search_index_name

# Create a search index and define the schema (names, types, and parameters)
index_client = SearchIndexClient(
    endpoint=cog_search_endpoint, credential=cog_search_cred)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="type", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SearchableField(name="tags", type=SearchFieldDataType.Collection(SearchFieldDataType.String),
                    filterable=True, searchable=True, retrievable=True),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

# Configure vector search
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

# Configure semantic search. This will allow us to conduct semantic or hybrid search (with vector search) later on if desired.
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        prioritized_keywords_fields=[SemanticField(field_name="type")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 opsbot-index created


![Our Azure Cognitive Search](./assets/MyAzureCognitiveSearch.png)
Here, we can see our index has been created.

## Create an indexer to pull data from Cosmos DB into Cognitive Search

Now we'll create the indexer, which will retrieve data from our Azure Cosmos DB resource. Learn more about Azure Cognitive Search Indexers [here](https://learn.microsoft.com/azure/search/search-howto-create-indexers)

In [33]:
def _create_datasource():
    # Here we create a datasource. 
    ds_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
    container = SearchIndexerDataContainer(name=cosmos_db_container_id)
    data_source_connection = SearchIndexerDataSourceConnection(
        name=cognitive_search_indexer_name, type="cosmosdb", connection_string=cosmosdb_connection_str, container=container
    )
    data_source = ds_client.create_or_update_data_source_connection(data_source_connection)
    return data_source

ds_name = _create_datasource().name

# Define indexer
indexer = SearchIndexer(
        name=cognitive_search_indexer_name,
        data_source_name=ds_name,
        target_index_name=index_name)

# Create indexer client
indexer_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)

# Create indexer
indexer_client.create_or_update_indexer(indexer)

result = indexer_client.get_indexer(cognitive_search_indexer_name)
print("Created indexer")

# Run the indexer we just created to ingest data into the search index.
indexer_client.run_indexer(result.name)


Created indexer


Now that we have setup our resources, data, and configured Azure Cognitive Search to index data from Azure Cosmos DB, let's try performing a vector similarity search.

In [34]:
# Function to assist with vector search
def vector_search(query):
    search_client = SearchClient(cog_search_endpoint, cognitive_search_index_name, cog_search_cred)  
    results = search_client.search(  
        search_text="",  
        vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
        select=["content", "type", "tags"] 
    )
    return results

Let's run a test query below.

In [50]:
query = "commerical card"  
results = vector_search(query)
for result in results:
    print(f"Title: {result['tags']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['type']}\n")  

Title: ["payments card","commercial card","card program","manage card","virtual cards"]
Score: 0.8772052
Content: . Commercial Card. Maximize your card strategy. Gain insights to develop and deliver a robust payments card strategy that can help expand your business’ potential. The WellsOne Commercial Card gives you comprehensive views of expenses, insights to program compliance, and cost management opportunities to inform stronger business decisions.. Consider the possibilities. A commercial card program lets you move your business-to-business payments from paper to electronic. Cards can help replace cumbersome employee expense reporting with on demand transaction entries, real-time visibility and streamlined review and approvals.  They can also reduce low-dollar purchase orders and check payments and improve security and control with virtual cards.. Optimization: Our supplier analysis and onboarding specialists are available to work with you and your suppliers to ensure your payments 

# Q&A over the data with GPT-3.5

Finally, we'll create a helper function to feed prompts into the `Completions` model. Then we'll create interactive loop where you can pose questions to the model and receive information grounded in your data.

In [51]:
#This function helps 

def generate_completion(prompt):
    print(prompt)

    system_prompt = '''
    You are an intelligent assistant for Wells Fargo Commerical Products
    You are designed to provide helpful answers to user questions about Wells Fargo Commerical Products given the information about to be provided.
        - Only answer questions related to the information provided below, provide 5 clear suggestions in a list format and bold the category
        - Write two lines of whitespace between each answer in the list.
        - Only provide answers that have products that are part of Wells Fargo Commerical Products.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users search themselves."
    '''

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]

    for i,item in enumerate(results_for_prompt):
        messages.append({"role": "system", "content": item['content']})

    response = openai.ChatCompletion.create(engine=completions_deployment, messages=messages)
    
    return response

In [ ]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about Wells Fargo Commerical Products. Type 'end' to end the session.\n")
user_input = input("Prompt: ")
while user_input.lower() != "end":
    results_for_prompt = vector_search(user_input)
    # print(results_for_prompt)
    completions_results = generate_completion(results_for_prompt)
    # print("\n")
    # print(completions_results)
    print(completions_results['choices'][0]['message']['content'])
    user_input = input("Prompt: ")